# Opcode Frequency

## Import and Const

In [1]:
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm

In [2]:
PATH = Path.cwd().parents[1]
DATA_PATH = os.path.join(PATH, 'data')
EXT_PATH = os.path.join(DATA_PATH, 'external')
INT_PATH = os.path.join(DATA_PATH, 'interim')

In [3]:
from evmdasm import EvmBytecode
from collections import Counter

def extract_opcode_frequency(hex_code):
    try:
        evm = EvmBytecode(bytecode=hex_code)
        instructions = evm.disassemble()
        # opcodes = [instr.name for instr in instructions if not (instr.name.startswith('INVALID') or instr.name.startswith('UNKNOWN'))]
        opcodes = [instr.name for instr in instructions]
        return Counter(opcodes)
    except Exception:
        return []

def extract_from_folder(folder_path):
    rows = []
    for filename in tqdm(os.listdir(folder_path)):
        if filename.endswith(".hex"):
            address = filename.replace(".hex", "")
        else:
            address = filename.replace(".txt", "")
        with open(os.path.join(folder_path, filename), "r") as f:
            bytecode = f.read().strip().lower().replace("0x", "")
        freq = extract_opcode_frequency(bytecode)
        freq["address"] = address.lower()
        rows.append(freq)

    df = pd.DataFrame(rows).fillna(0).astype({k: int for k in rows[0] if k != 'address'})
    cols = ['address'] + [col for col in df.columns if col != 'address']
    df = df[cols]
    return df

In [4]:
data = []

## CRPWarner

In [5]:
CRPWARNER_PATH = os.path.join(EXT_PATH, 'crpwarner')

### Ground Truth

In [6]:
GROUND_PATH = os.path.join(CRPWARNER_PATH, 'groundtruth/hex')

In [7]:
df = extract_from_folder(GROUND_PATH)

  3%|▎         | 2/72 [00:00<00:11,  6.22it/s]invalid instruction: PUSH30


 33%|███▎      | 24/72 [00:04<00:07,  6.19it/s]invalid instruction: PUSH13
invalid instruction: PUSH25
 58%|█████▊    | 42/72 [00:07<00:04,  7.15it/s]invalid instruction: PUSH29
invalid instruction: PUSH29
 67%|██████▋   | 48/72 [00:07<00:02, 10.19it/s]invalid instruction: PUSH20
invalid instruction: PUSH17
 72%|███████▏  | 52/72 [00:08<00:02,  9.37it/s]invalid instruction: PUSH20
invalid instruction: PUSH13
 96%|█████████▌| 69/72 [00:10<00:00,  9.52it/s]invalid instruction: PUSH27
invalid instruction: PUSH32
100%|██████████| 72/72 [00:10<00:00,  6.83it/s]


In [8]:
df.head()

,address,PUSH1,MSTORE,CALLDATASIZE,LT,PUSH2,JUMPI,CALLDATALOAD,PUSH29,SWAP1,...,UNKNOWN_0xc6,UNKNOWN_0xe1,INVALID_0x70,PUSH30,DUP16,UNKNOWN_0x2b,UNKNOWN_0xd8,INVALID_0x7a,UNKNOWN_0xf9,INVALID_0x7f
0,0x8275ebf521dc217aa79c88132017a5bcef001dd9,386,112,10,7,126,53,17,1,209,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0x292e89d5d5bdab3af2f5838c194c1983f0140b43,413,115,10,8,188,63,7,0,89,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0x2753dce37a7edb052a77832039bcc9aa49ad8b25,660,176,23,13,378,101,7,0,157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0xe1a0ce8b94c6a5e4791401086763d7bd0a6c18f5,264,77,8,13,116,36,15,0,103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0x9db8a10c7fe60d84397860b3af2e686d4f90c2b7,801,248,20,29,312,100,35,0,310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 266 entries, address to INVALID_0x7f
dtypes: float64(199), int64(66), object(1)
memory usage: 149.8+ KB


In [10]:
data.append(df)

In [11]:
OUT_PATH = os.path.join(INT_PATH, 'crpwarner')

In [12]:
df.to_csv(os.path.join(OUT_PATH, 'groundtruth-feature.csv'), index=False)

### Large

In [13]:
LARGE_PATH = os.path.join(CRPWARNER_PATH, 'large/hex')

In [14]:
df = extract_from_folder(LARGE_PATH)

  0%|          | 0/13483 [00:00<?, ?it/s]

  0%|          | 7/13483 [00:00<22:34,  9.95it/s]invalid instruction: PUSH30
invalid instruction: PUSH22
  0%|          | 18/13483 [00:02<30:48,  7.28it/s]invalid instruction: PUSH21
invalid instruction: PUSH17
  0%|          | 32/13483 [00:03<23:27,  9.55it/s]invalid instruction: PUSH28
invalid instruction: PUSH16
  0%|          | 34/13483 [00:03<20:03, 11.18it/s]invalid instruction: PUSH24
invalid instruction: PUSH11
  0%|          | 55/13483 [00:18<4:36:23,  1.24s/it] invalid instruction: PUSH32
invalid instruction: PUSH22
  0%|          | 64/13483 [00:19<50:40,  4.41it/s]  invalid instruction: PUSH19
invalid instruction: PUSH16
  1%|          | 68/13483 [00:20<27:46,  8.05it/s]invalid instruction: PUSH12
invalid instruction: PUSH29
  1%|          | 76/13483 [00:20<17:10, 13.01it/s]invalid instruction: PUSH18
invalid instruction: PUSH32
  1%|          | 78/13483 [00:20<21:21, 10.46it/s]invalid instruction: PUSH31
invalid instruction: PUSH30
invalid instruction: PUSH29
  1%|         

In [15]:
df.head()

,address,PUSH1,MSTORE,CALLDATASIZE,LT,PUSH2,JUMPI,CALLDATALOAD,PUSH29,SWAP1,...,UNKNOWN_0xe6,INVALID_0x68,INVALID_0x6d,INVALID_0x67,INVALID_0x69,INVALID_0x64,INVALID_0x66,INVALID_0x73,INVALID_0x62,INVALID_0x65
0,0x71a982a028c9d4b0566041a78df12b810462e155,417,103,15,10,156,75,27,2,227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0x9b11efcaaa1890f6ee52c6bb7cf8153ac5d74139,304,76,1,7,135,63,14,1,163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0x9ec8d44af808d7cca2ec23c0dc0d1f49a3386ea4,462,106,15,25,173,92,27,2,232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0x662abcad0b7f345ab7ffb1b1fbb9df7894f18e66,334,97,1,9,121,56,17,1,191,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0x21f15966e07a10554c364b988e91dab01d32794a,711,172,16,19,212,107,35,1,375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13483 entries, 0 to 13482
Columns: 287 entries, address to INVALID_0x65
dtypes: float64(216), int64(70), object(1)
memory usage: 29.5+ MB


In [17]:
data.append(df)

In [18]:
df.to_csv(os.path.join(OUT_PATH, 'large-feature.csv'), index=False)

## RPHunter

In [19]:
RP_PATH = os.path.join(EXT_PATH, 'rphunter')

### Normal

In [20]:
NOR_PATH = os.path.join(RP_PATH, 'Normal-Bytecode')

In [21]:
df = extract_from_folder(NOR_PATH)

  0%|          | 0/1675 [00:00<?, ?it/s]invalid instruction: PUSH20
invalid instruction: PUSH30
  0%|          | 3/1675 [00:00<01:09, 23.96it/s]invalid instruction: PUSH16
invalid instruction: PUSH13
invalid instruction: PUSH26
  0%|          | 7/1675 [00:00<00:54, 30.57it/s]invalid instruction: PUSH17
invalid instruction: PUSH15
  1%|          | 12/1675 [00:00<00:59, 27.98it/s]invalid instruction: PUSH29
invalid instruction: PUSH32
invalid instruction: PUSH16
  1%|          | 17/1675 [00:00<00:55, 29.84it/s]invalid instruction: PUSH7
invalid instruction: PUSH31
  2%|▏         | 26/1675 [00:00<00:57, 28.80it/s]invalid instruction: PUSH21
invalid instruction: PUSH24
  2%|▏         | 29/1675 [00:01<01:12, 22.61it/s]invalid instruction: PUSH27
invalid instruction: PUSH24
invalid instruction: PUSH32
  2%|▏         | 32/1675 [00:01<01:47, 15.25it/s]invalid instruction: PUSH16
invalid instruction: PUSH27
invalid instruction: PUSH26
invalid instruction: PUSH12
  2%|▏         | 36/1675 [00:01<

In [22]:
df.head()

,address,PUSH1,MSTORE,CALLDATASIZE,LT,PUSH2,JUMPI,CALLDATALOAD,PUSH29,SWAP1,...,INVALID_0x76,UNKNOWN_0xe1,PUSH30,INVALID_0x64,INVALID_0x62,INVALID_0x7b,INVALID_0x6d,INVALID_0x63,PUSH31,INVALID_0x67
0,0x6b466b0232640382950c45440ea5b630744eca99,293,84,7,11,92,47,15,3,155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0x4e15361fd6b4bb609fa63c81a2be19d873717870,1039,205,4,47,524,254,67,1,358,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0xa95c4f2e0d6455637f67f655da4afae5d50d859b,350,98,7,12,112,54,15,3,188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0x35dd2ebf20746c6e658fac75cd80d4722fae62f6,316,98,9,14,115,52,16,1,162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0x264dc2dedcdcbb897561a57cba5085ca416fb7b4,379,104,1,10,120,58,20,2,181,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Columns: 287 entries, address to INVALID_0x67
dtypes: float64(216), int64(70), object(1)
memory usage: 3.7+ MB


In [24]:
data.append(df)

In [25]:
OUT_PATH = os.path.join(INT_PATH, 'rphunter')

In [26]:
df.to_csv(os.path.join(OUT_PATH, 'normal-feature.csv'), index=False)

### Rug 

In [27]:
RUG_PATH = os.path.join(RP_PATH, 'Rug-Bytecode')

In [28]:
df = extract_from_folder(RUG_PATH)

  0%|          | 0/652 [00:00<?, ?it/s]

  0%|          | 3/652 [00:00<00:24, 26.13it/s]invalid instruction: PUSH16
invalid instruction: PUSH16
  3%|▎         | 18/652 [00:00<00:32, 19.30it/s]invalid instruction: PUSH25
invalid instruction: PUSH32
  4%|▎         | 23/652 [00:01<00:40, 15.66it/s]invalid instruction: PUSH28
invalid instruction: PUSH22
invalid instruction: PUSH30
  5%|▌         | 35/652 [00:02<00:45, 13.63it/s]invalid instruction: PUSH13
invalid instruction: PUSH28
invalid instruction: PUSH16
  6%|▋         | 42/652 [00:02<00:30, 20.25it/s]invalid instruction: PUSH24
invalid instruction: PUSH15
  7%|▋         | 45/652 [00:02<00:28, 21.14it/s]invalid instruction: PUSH32
invalid instruction: PUSH12
  8%|▊         | 51/652 [00:02<00:30, 19.58it/s]invalid instruction: PUSH29
invalid instruction: PUSH13
invalid instruction: PUSH26
  8%|▊         | 54/652 [00:02<00:27, 21.76it/s]invalid instruction: PUSH29
invalid instruction: PUSH13
invalid instruction: PUSH26
  9%|▉         | 61/652 [00:03<00:28, 20.81it/s]invalid i

In [29]:
df.head()

,address,PUSH1,MSTORE,CALLVALUE,DUP1,ISZERO,PUSH2,JUMPI,REVERT,JUMPDEST,...,UNKNOWN_0xcb,INVALID_0x74,UNKNOWN_0xae,UNKNOWN_0xe4,INVALID_0x7e,INVALID_0x6d,INVALID_0x72,INVALID_0x69,INVALID_0x68,INVALID_0x76
0,0x6d86f0a41c3966cef8ea139648db707e912563c9,589,175,1,114,48,306,89,27,195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0xd248537d601f8e062dd36c8e7d26302d51cc653a,851,255,1,347,67,316,101,38,224,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0xa4dbc813f7e1bf5827859e278594b1e0ec1f710f,729,180,28,202,71,382,130,61,246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0xc606b9ac3dc335aba427011547a4211c298ca5e4,1980,451,60,681,238,840,315,181,538,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0xc5a25e92e691635bdd6df2e904633dc3152360cd,949,175,31,298,144,489,183,93,318,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Columns: 281 entries, address to INVALID_0x76
dtypes: float64(199), int64(81), object(1)
memory usage: 1.4+ MB


In [31]:
data.append(df)

In [32]:
df.to_csv(os.path.join(OUT_PATH, 'rug-feature.csv'), index=False)

## Trapdoor

In [33]:
TRAP_PATH = os.path.join(EXT_PATH, 'trapdoordata/hex')

In [34]:
df = extract_from_folder(TRAP_PATH)

  0%|          | 2/11943 [00:00<10:19, 19.28it/s]

invalid instruction: PUSH32
  0%|          | 16/11943 [00:01<17:38, 11.27it/s]invalid instruction: PUSH16
invalid instruction: PUSH22
  0%|          | 27/11943 [00:03<27:41,  7.17it/s]invalid instruction: PUSH31
invalid instruction: PUSH21
  0%|          | 30/11943 [00:03<19:45, 10.05it/s]invalid instruction: PUSH29
invalid instruction: PUSH20
  0%|          | 40/11943 [00:04<22:03,  9.00it/s]invalid instruction: PUSH20
invalid instruction: PUSH32
  1%|          | 76/11943 [00:08<19:21, 10.21it/s]invalid instruction: PUSH29
invalid instruction: PUSH27
  1%|          | 93/11943 [00:09<15:55, 12.41it/s]invalid instruction: PUSH16
invalid instruction: PUSH22
  1%|          | 96/11943 [00:09<13:42, 14.40it/s]invalid instruction: PUSH13
invalid instruction: PUSH20
  1%|          | 102/11943 [00:10<18:40, 10.57it/s]invalid instruction: PUSH20
invalid instruction: PUSH13
  1%|          | 123/11943 [00:12<12:58, 15.18it/s]invalid instruction: PUSH24
invalid instruction: PUSH20
  1%|          |

In [35]:
df.head()

,address,PUSH1,MSTORE,CALLDATASIZE,LT,PUSH2,JUMPI,CALLDATALOAD,SHR,DUP1,...,INVALID_0x71,INVALID_0x79,INVALID_0x72,INVALID_0x76,PUSH30,INVALID_0x78,INVALID_0x6e,INVALID_0x6d,INVALID_0x6b,INVALID_0x6a
0,0x0983118867e36dee0f993bbfbe7f67639e2db3c9,776,168,11,14,381,125,13,1,208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0x32e1c61e6213a101d474404a74dca27c25f8f630,380,93,9,15,158,74,17,1,213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0x74d6a669e27ac5cfa6bc380f983e5d82aef66f5a,918,194,10,9,394,133,11,1,213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0xe5ffd337213d7f565721bb3abbf539efb8756031,1911,508,32,38,927,329,25,2,448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0x7f0c3fb8755c3ee81f2c6f20f152bc27b0662348,1336,288,28,19,723,271,17,2,367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11943 entries, 0 to 11942
Columns: 279 entries, address to INVALID_0x6a
dtypes: float64(187), int64(91), object(1)
memory usage: 25.4+ MB


In [37]:
data.append(df)

In [38]:
OUT_PATH = os.path.join(INT_PATH, 'trapdoor')

In [39]:
df.to_csv(os.path.join(OUT_PATH, 'feature.csv'), index=False)

## Show Check Feature

In [40]:
# Convert column sets
col_sets = [set(df.columns) for df in data]

# 1. Common columns
common_cols = set.intersection(*col_sets)

# 2. Number of columns in each
col_counts = {i: len(cols) for i, cols in enumerate(col_sets)}

# 3. Differences (unique to each)
diffs = {
    i: cols - common_cols
    for i, cols in enumerate(col_sets)
}

# Display results
print(f"✅ Common Columns ({len(common_cols)}):\n{sorted(common_cols)}\n")
print("📊 Column Counts:")
for name, count in col_counts.items():
    print(f"  {name}: {count}")

print("\n❌ Differences (unique columns):")
for name, unique in diffs.items():
    print(f"  {name} ({len(unique)} unique): {sorted(unique)}")

✅ Common Columns (266):
['ADD', 'ADDMOD', 'ADDRESS', 'AND', 'BALANCE', 'BASEFEE', 'BLOCKHASH', 'BYTE', 'CALL', 'CALLCODE', 'CALLDATACOPY', 'CALLDATALOAD', 'CALLDATASIZE', 'CALLER', 'CALLVALUE', 'CHAINID', 'CODECOPY', 'CODESIZE', 'COINBASE', 'CREATE', 'CREATE2', 'DELEGATECALL', 'DIFFICULTY', 'DIV', 'DUP1', 'DUP10', 'DUP11', 'DUP12', 'DUP13', 'DUP14', 'DUP15', 'DUP16', 'DUP2', 'DUP3', 'DUP4', 'DUP5', 'DUP6', 'DUP7', 'DUP8', 'DUP9', 'EQ', 'EXP', 'EXTCODECOPY', 'EXTCODEHASH', 'EXTCODESIZE', 'GAS', 'GASLIMIT', 'GASPRICE', 'GT', 'INVALID_0x6c', 'INVALID_0x6e', 'INVALID_0x6f', 'INVALID_0x70', 'INVALID_0x72', 'INVALID_0x73', 'INVALID_0x74', 'INVALID_0x77', 'INVALID_0x78', 'INVALID_0x79', 'INVALID_0x7a', 'INVALID_0x7c', 'INVALID_0x7d', 'INVALID_0x7e', 'INVALID_0x7f', 'ISZERO', 'JUMP', 'JUMPDEST', 'JUMPI', 'LOG0', 'LOG1', 'LOG2', 'LOG3', 'LOG4', 'LT', 'MLOAD', 'MOD', 'MSIZE', 'MSTORE', 'MSTORE8', 'MUL', 'MULMOD', 'NOT', 'NUMBER', 'OR', 'ORIGIN', 'PC', 'POP', 'PUSH1', 'PUSH10', 'PUSH11', 'PUSH12'